In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook as tqdm

In [ ]:
# lees bestanden gt
# connvert file names naar OCR
# lees ocr bestanden
import os

from nlppln.utils import get_files

gs_dir = '/home/jvdzwaan/ownCloud/Shared/OCR/Ground-truth/'
ocr_dir = '/home/jvdzwaan/ownCloud/Shared/OCR/Originele ALTOs/'


def gt_fname2ocr_fname(fname):
    bn = os.path.basename(fname)
    return bn.replace('GT', 'alto')

gs_files = get_files(gs_dir)
print(len(gs_files))
print([f for f in gs_files if 'extra' in f])
# remove file with "extra" in the name, this one is the same as the file without "extra" in the name
gs_files = [f for f in gs_files if not 'extra' in f]
print(len(gs_files))
gt_fname2ocr_fname(gs_files[0])

In [ ]:
ocr_files = []
for gs_file in gs_files:
    ocr_bn = gt_fname2ocr_fname(gs_file)
    ocr_file = os.path.join(ocr_dir, ocr_bn)
    if os.path.isfile(ocr_file):
        ocr_files.append(ocr_file)
    else:
        print('File not found:', ocr_file)
        print('GS file:', gs_file)
print(len(ocr_files))

In [ ]:
from collections import OrderedDict

from lxml import etree

def get_words_in_textlines(fname, alto_ns):
    lines = OrderedDict()
    context = etree.iterparse(fname, events=('end', ), tag=(alto_ns+'TextLine'))
    for event, elem in context:
        lines[elem.attrib['ID']] = []
        for a in elem.getchildren():
            if a.tag == alto_ns+'String':
                if a.attrib.get('SUBS_TYPE') == 'HypPart1':
                    lines[elem.attrib['ID']].append(a.attrib['SUBS_CONTENT'])
                elif a.attrib.get('SUBS_TYPE') != 'HypPart2':
                    lines[elem.attrib['ID']].append(a.attrib['CONTENT'])
                    
        #for a in elem.getchildren():
        #    if a.tag == alto_ns+'String':
        #        lines[elem.attrib['ID']].append(a.attrib['CONTENT'])
        
         # make iteration over context fast and consume less memory
        #https://www.ibm.com/developerworks/xml/library/x-hiperfparse
        elem.clear()
        while elem.getprevious() is not None:
            del elem.getparent()[0]
            
    return lines